In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy model in GCS and Serving it with Text Generation Inference (TGI) on Vertex AI

> [meta-llama/Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)

> [Import a model programmatically](https://cloud.google.com/vertex-ai/docs/model-registry/import-model#import_a_model_programmatically)

In [1]:
# @title Install Vertex AI SDK and other required packages
%pip install --upgrade --user --quiet google-cloud-aiplatform \
                                      huggingface_hub \
                                      transformers

In [2]:
# @title Define project information
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

In [3]:
# @title GCP Authentication

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)

In [4]:
# @title Authenticate your Hugging Face account
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) y


In [4]:
# @title Check Project ID and Location
import os

PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

PROJECT_ID, LOCATION

('ai-hangsik', 'us-central1')

## Requirements

You will need to have the following IAM roles set:

- Artifact Registry Reader (roles/artifactregistry.reader)
- Vertex AI User (roles/aiplatform.user)

For more information about granting roles, see [Manage access](https://cloud.google.com/iam/docs/granting-changing-revoking-access).

---

You will also need to enable the following APIs (if not enabled already):

- Vertex AI API (aiplatform.googleapis.com)
- Artifact Registry API (artifactregistry.googleapis.com)

For more information about API enablement, see [Enabling APIs](https://cloud.google.com/apis/docs/getting-started#enabling_apis).

---

To access Llama on Hugging Face, you’re required to review and agree to the model usage license on the Hugging Face Hub for any of the models from the [meta-llama/Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct), and the access request will be processed inmediately.

In [5]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable artifactregistry.googleapis.com

In [ ]:
# from google.cloud import aiplatform

# def upload_model_from_gcs(project_id, model_display_name, artifact_uri, serving_container_image_uri):
#   """Uploads a model to Vertex AI from a GCS location.

#   Args:
#     project_id: Your Google Cloud project ID.
#     model_display_name: The display name for the uploaded model.
#     artifact_uri: The GCS URI where the model artifact is stored.
#     serving_container_image_uri: The URI of the serving container image.
#   """

#   aiplatform.init(project=project_id, location='us-central1')  # Replace with your region

#   model = aiplatform.Model.upload(
#       display_name=model_display_name,
#       artifact_uri=artifact_uri,
#       serving_container_image_uri=serving_container_image_uri,
#       sync=True  # Wait for the upload to complete
#   )

#   print(f"Model uploaded: {model.resource_name}")
#   return model

In [7]:
# @title Published TGI Containers
!gcloud container images list --repository="us-docker.pkg.dev/deeplearning-platform-release/gcr.io" | grep "huggingface-text-generation-inference"

us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.1-4.ubuntu2204.py310
us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.2-0.ubuntu2204.py310
us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.2-1.ubuntu2204.py310
us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.2-2.ubuntu2204.py310
us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu124.2-3.ubuntu2204.py311
us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu124.2-4.ubuntu2204.py311


In [5]:
# @title Upload model from GCS
from google.cloud import aiplatform

project_id = "ai-hangsik"  # Replace with your project ID
model_display_name = "Llama-3.1-8B-Instruct_gcs"
artifact_uri = "gs://sllm_0106/model"  # Replace with your GCS URI
serving_container_image_uri="us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.2-2.ubuntu2204.py310"
#serving_container_image_uri="us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu124.2-4.ubuntu2204.py311"

aiplatform.init(project=project_id, location='us-central1')  # Replace with your region

model = aiplatform.Model.upload(
    display_name=model_display_name,
    artifact_uri=artifact_uri,
    serving_container_image_uri=serving_container_image_uri,
    serving_container_environment_variables={
            "MODEL_ID": "meta-llama/Llama-3.1-8B-Instruct",
            "NUM_SHARD": "1",
            "MAX_INPUT_TOKENS": "512",
            "MAX_TOTAL_TOKENS": "1024",
            "MAX_BATCH_PREFILL_TOKENS": "1512",
        }
)

print(f"Model uploaded: {model.resource_name}")


INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/721521243942/locations/us-central1/models/6947564386777038848/operations/4051683752600928256
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/721521243942/locations/us-central1/models/6947564386777038848@1
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/721521243942/locations/us-central1/models/6947564386777038848@1')


Model uploaded: projects/721521243942/locations/us-central1/models/6947564386777038848


In [6]:
deployed_model = model.deploy(
    endpoint=aiplatform.Endpoint.create(display_name="Llama-3.1-8B-Instruct_gcs"),
    machine_type="g2-standard-4",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/721521243942/locations/us-central1/endpoints/9197169575253245952/operations/8732049665345716224
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/721521243942/locations/us-central1/endpoints/9197169575253245952
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/721521243942/locations/us-central1/endpoints/9197169575253245952')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/721521243942/locations/us-central1/endpoints/9197169575253245952
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/721521243942/locations/us-central1/endpoints/9197169575253245952/operations/6079429484824494080


FailedPrecondition: 400 Model server exited unexpectedly. Model server logs can be found at https://console.cloud.google.com/logs/viewer?project=721521243942&resource=aiplatform.googleapis.com%2FEndpoint&advancedFilter=resource.type%3D%22aiplatform.googleapis.com%2FEndpoint%22%0Aresource.labels.endpoint_id%3D%229197169575253245952%22%0Aresource.labels.location%3D%22us-central1%22. 9: Model server exited unexpectedly. Model server logs can be found at https://console.cloud.google.com/logs/viewer?project=721521243942&resource=aiplatform.googleapis.com%2FEndpoint&advancedFilter=resource.type%3D%22aiplatform.googleapis.com%2FEndpoint%22%0Aresource.labels.endpoint_id%3D%229197169575253245952%22%0Aresource.labels.location%3D%22us-central1%22.

## Online predictions on Vertex AI

Once the model is deployed on Vertex AI, you can run the online predictions using the `aiplatform.Endpoint.predict` method, which will send the requests to the running endpoint in the `/predict` route specified within the container following Vertex AI I/O payload formatting.

As you are serving a `text-generation` model fine-tuned for instruction-following, you will need to make sure that the chat template, if any, is applied correctly to the input conversation; meaning that `transformers` need to be installed so as to instantiate the `tokenizer` for [`google/gemma-7b-it`](https://huggingface.co/google/gemma-7b-it) and run the `apply_chat_template` method over the input conversation before sending the input within the payload to the Vertex AI endpoint.

> Note that the Messages API will be supported on Vertex AI on upcoming TGI releases, starting on 2.3, meaning that at the time of writing this post, the prompts need to be formatted before sending the request if you want to achieve nice results (assuming you are using an intruction-following model and not a base model).

In [ ]:
%pip install --upgrade --user --quiet transformers jinja2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.6/134.6 kB 2.7 MB/s eta 0:00:00


After the installation is complete, the following snippet will apply the chat template to the conversation:

In [ ]:
import os
from huggingface_hub import get_token
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    token=get_token(),
)

messages = [
    {"role": "system", "content": "You are an assistant that responds as an AI expert."},
    {"role": "user", "content": "What's Deep learning?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

In [ ]:
output = deployed_model.predict(
    instances=[
        {
            "inputs": "What's Deep Learning? in details",
            "parameters": {
                "max_new_tokens": 128,
                "do_sample": True,
                "top_p": 0.95,
                "temperature": 0.7,
            },
        },
    ]
)

output


Prediction(predictions=['Deep learning is a subfield of machine learning (ML) that specializes in artificial neural networks with multiple layers. These networks are modeled after the human brain\'s neural structures and are capable of learning complex patterns in data.\n\nIn a traditional neural network, the data flows through a series of interconnected nodes or "neurons," which perform computations to transform the input data into a predicted output. The architecture of deep neural networks involves numerous hidden layers of these nodes, with each layer learning representations of the data from the previous layer.\n\nDeep learning models are particularly useful for image and speech recognition tasks. Some of the key characteristics of deep learning include:\n\n1.'], deployed_model_id='6301157102761017344', metadata=None, model_version_id='1', model_resource_name='projects/721521243942/locations/us-central1/models/704449403334688768', explanations=None)

Which is what you will be sending within the payload to the deployed Vertex AI Endpoint, as well as [the generation parameters](https://huggingface.co/docs/huggingface_hub/main/en/package_reference/inference_client#huggingface_hub.InferenceClient.text_generation).

#### From a different session

To run the online prediction from a different session, you can run the following snippet.

In [ ]:
import os

from google.cloud import aiplatform


aiplatform.init(project=PROJECT_ID, location=LOCATION)

endpoint_display_name = (
    "Llama-3.1-8B-Instruct"  # TODO: change to your endpoint display name
)


# Iterates over all the Vertex AI Endpoints within the current project and keeps the first match (if any), otherwise set to None
ENDPOINT_ID = next(
    (
        endpoint.name
        for endpoint in aiplatform.Endpoint.list()
        if endpoint.display_name == endpoint_display_name
    ),
    None,
)
assert ENDPOINT_ID, (
    "`ENDPOINT_ID` is not set, please make sure that the `endpoint_display_name` is correct at "
    f"https://console.cloud.google.com/vertex-ai/online-prediction/endpoints?project={os.getenv('PROJECT_ID')}"
)

endpoint = aiplatform.Endpoint(
    f"projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}"
)
output = endpoint.predict(
    instances=[
        {
            "inputs": "<bos><start_of_turn>user\nWhat's Deep Learning? in details<end_of_turn>\n<start_of_turn>model\n",
            "parameters": {
                "max_new_tokens": 128,
                "do_sample": True,
                "top_p": 0.95,
                "temperature": 0.7,
            },
        },
    ],
)
output

Prediction(predictions=["Deep learning is a subset of machine learning that utilizes neural networks to analyze and learn from data. In essence, it's a type of artificial intelligence (AI) that is modeled after the human brain's functions of vision, speech, pattern recognition, and decision-making.\n\nHere's a more detailed explanation:\n\n1. **Neural Networks**: Neural networks are the base of deep learning. They consist of layers of interconnected nodes (neurons) that process and transmit information. Each node receives input from the previous layer, performs a computation, and then sends the output to the next layer.\n2. **Hierarchical Representation**: Deep learning models consist of"], deployed_model_id='6301157102761017344', metadata=None, model_version_id='1', model_resource_name='projects/721521243942/locations/us-central1/models/704449403334688768', explanations=None)

### Via gcloud

You can also send the requests using the `gcloud` CLI via the `gcloud ai endpoints` command.

> Note that, before proceeding, you should either replace the values or set the following environment variables in advance from the Python variables set in the example, as follows:
>
> ```python
> import os
> os.environ["PROJECT_ID"] = PROJECT_ID
> os.environ["LOCATION"] = LOCATION
> os.environ["ENDPOINT_NAME"] = "google--gemma-7b-it-endpoint"
> ```

In [ ]:
%%bash
ENDPOINT_ID=$(gcloud ai endpoints list \
  --project=$PROJECT_ID \
  --region=$LOCATION \
  --filter="display_name=$ENDPOINT_NAME" \
  --format="value(name)" \
  | cut -d'/' -f6)

echo '{
  "instances": [
    {
      "inputs": "<bos><start_of_turn>user\nWhat'\''s Deep Learning?<end_of_turn>\n<start_of_turn>model\n",
      "parameters": {
        "max_new_tokens": 20,
        "do_sample": true,
        "top_p": 0.95,
        "temperature": 1.0
      }
    }
  ]
}' | gcloud ai endpoints predict $ENDPOINT_ID \
  --project=$PROJECT_ID \
  --region=$LOCATION \
  --json-request="-"

### Via cURL

Alternatively, you can also send the requests via `cURL`.

> Note that, before proceeding, you should either replace the values or set the following environment variables in advance from the Python variables set in the example, as follows:
>
> ```python
> import os
> os.environ["PROJECT_ID"] = PROJECT_ID
> os.environ["LOCATION"] = LOCATION
> os.environ["ENDPOINT_NAME"] = "google--gemma-7b-it-endpoint"
> ```

In [ ]:
%%bash
ENDPOINT_ID=$(gcloud ai endpoints list \
  --project=$PROJECT_ID \
  --region=$LOCATION \
  --filter="display_name=$ENDPOINT_NAME" \
  --format="value(name)" \
  | cut -d'/' -f6)

curl -X POST \
    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
    -H "Content-Type: application/json" \
    https://$LOCATION-aiplatform.googleapis.com/v1/projects/$PROJECT_ID/locations/$LOCATION/endpoints/$ENDPOINT_ID:predict \
    -d '{
        "instances": [
            {
                "inputs": "<bos><start_of_turn>user\nWhat'\''s Deep Learning?<end_of_turn>\n<start_of_turn>model\n",
                "parameters": {
                    "max_new_tokens": 20,
                    "do_sample": true,
                    "top_p": 0.95,
                    "temperature": 1.0
                }
            }
        ]
    }'

## Cleaning up

Finally, you can already release the resources that you've created as follows, to avoid unnecessary costs:

- `deployed_model.undeploy_all` to undeploy the model from all the endpoints.
- `deployed_model.delete` to delete the endpoint/s where the model was deployed gracefully, after the `undeploy_all` method.
- `model.delete` to delete the model from the registry.

In [ ]:
deployed_model.undeploy_all()
deployed_model.delete()
model.delete()

Alternatively, you can also remove those from the Google Cloud Console following the steps:

- Go to Vertex AI in Google Cloud
- Go to Deploy and use -> Online prediction
- Click on the endpoint and then on the deployed model/s to "Undeploy model from endpoint"
- Then go back to the endpoint list and remove the endpoint
- Finally, go to Deploy and use -> Model Registry, and remove the model

## References

- [GitHub Repository - Hugging Face DLCs for Google Cloud](https://github.com/huggingface/Google-Cloud-Containers): contains all the containers developed by the collaboration of both Hugging Face and Google Cloud teams; as well as a lot of examples on both training and inference, covering both CPU and GPU, as well support for most of the models within the Hugging Face Hub.
- [Google Cloud Documentation - Hugging Face DLCs](https://cloud.google.com/deep-learning-containers/docs/choosing-container#hugging-face): contains a table with the latest released Hugging Face DLCs on Google Cloud.
- [Google Artifact Registry - Hugging Face DLCs](https://console.cloud.google.com/artifacts/docker/deeplearning-platform-release/us/gcr.io): contains all the DLCs released by Google Cloud that can be used.
- [Hugging Face Documentation - Google Cloud](https://huggingface.co/docs/google-cloud): contains the official Hugging Face documentation for the Google Cloud DLCs.